In [1]:
import pandas as pd
from RCH_module.RCH import get_volumes

In [2]:
xls = pd.ExcelFile('input_total/2024 12 03 Viajes almacen bcn solo mercancia pasada por volumétrica con clonados, ADR y Aduanas.xlsx')
viajes_df = pd.read_excel(xls, 'Viajes')
partidas_df = pd.read_excel(xls, 'Partidas')


In [3]:
partidas_df['Remontable'] = partidas_df['Remontable'].map({'NO': 0, 'SI': 1})

In [4]:
viajes_df.loc[viajes_df['TipoEquipo'] == "MAXI 45'", 'AltoCm'] = 259
viajes_df.loc[viajes_df['TipoEquipo'] == "MAXI 45'", 'LargoCm'] = 1350
viajes_df.loc[viajes_df['TipoEquipo'] == "MAXI 45'", 'AnchoCm'] = 244

viajes_df = viajes_df.drop(['TipoEquipoAltoMetros', 'TipoEquipoAnchoMetros', 'TipoEquipoLongitudMetros'], axis=1)

In [5]:
partidas_df['Volumen'] = partidas_df['AltoCm']*partidas_df['AnchoCm']*partidas_df['LargoCm']


In [6]:
volumen_total_por_viaje = partidas_df.groupby(by=['CodigoViaje'], as_index=False)['Volumen'].agg('sum')
volumen_total_por_viaje.rename(columns={'Volumen': 'VolumenCargado'}, inplace=True)
viajes_df = viajes_df.merge(volumen_total_por_viaje, how='left', on='CodigoViaje')

viajes_df['VolumenMax'] = viajes_df['AltoCm']*viajes_df['AnchoCm']*viajes_df['LargoCm']
viajes_df['Volumen%'] = viajes_df['VolumenCargado']/viajes_df['VolumenMax'] * 100 

In [ ]:
viajes_ultima_semana = viajes_df[(viajes_df['TipoEquipo'] == "MAXI 45'") & (viajes_df['FechaCierreViaje'] > '2024-12-10') & (viajes_df['PuroVolumetrica'] == 1)]
viajes_prueba = viajes_ultima_semana[['CodigoViaje', 'NumPartidas', 'Volumen%']].copy()
viajes_prueba = viajes_prueba.reset_index()
len(viajes_ultima_semana)

46

In [27]:
viaje = viajes_prueba.CodigoViaje.iloc[0]

In [28]:
test_df = partidas_df[partidas_df['CodigoViaje'] == viaje]

# Guardar el archivo con un nombre único por viaje
test_df.to_excel(f'viajes_prueba/test_{viaje}.xlsx', index=False)

avg_volume, best_floor, best_volume, not_loaded = get_volumes(viaje, load_type=1, file_path=f'viajes_prueba/test_{viaje}.xlsx')

viajes_prueba.loc[viajes_prueba['CodigoViaje'] == viaje, 'AverageVolume'] = avg_volume
viajes_prueba.loc[viajes_prueba['CodigoViaje'] == viaje, 'BestFloorV'] = best_floor[0]
viajes_prueba.loc[viajes_prueba['CodigoViaje'] == viaje, 'BestFloorF'] = best_floor[1]
viajes_prueba.loc[viajes_prueba['CodigoViaje'] == viaje, 'BestVolumeV'] = best_volume[0]
viajes_prueba.loc[viajes_prueba['CodigoViaje'] == viaje, 'BestVolumeF'] = best_volume[1]

viajes_prueba.loc[viajes_prueba['CodigoViaje'] == viaje, 'NotLoaded'] = not_loaded

Scores:  (69.71481353595175, 93.69286359530263, 1337)
Not loaded:  0
Scores:  (69.71481353595175, 92.71002710027099, 1321)
Not loaded:  0
Scores:  (69.71029333630959, 93.4977416440831, 1321)
Not loaded:  1
Scores:  (69.623258566348, 92.11201445347787, 1257)
Not loaded:  1
Scores:  (69.623258566348, 92.11201445347787, 1332)
Not loaded:  1
